# ADC Crash Course!

An attempt to aggregate a bunch of lessons learned items from several decades of ADC applications.

This notebook is intended to be accessible from several angles:
Running Jupyter Notebook locally
Running in Google Colab - no istallation of anything needed!

Either of these can run the simulation cells, and dependencies will be kept to a minimum.

There will also be examples that talk to hardware - this requires that the local runtime environment be set up properly with all dependencies. Notes are included below on how to connect to the local runtime from Google Colab as well.


**References to organize later:**

1. Maximum SNR vs. Clock Jitter:
https://www.analog.com/en/technical-articles/maximum-snr-vs-clock-jitter.html

2. Sampled Systems and the Effects of Clock Phase Noise and Jitter:
http://www.analog.com/AN-756

3. On the Use of Windows for Harmonic Analysis With the Discrete Fourier Transform, Frederic Harris
https://www.researchgate.net/publication/2995027_On_the_Use_of_Windows_for_Harmonic_Analysis_With_the_Discrete_Fourier_Transform
4. "Taking the Mystery "SNR = 6.02N + 1.76dB," and Why You Should Care" (EVEN THOUGH with most modern converters you shouldn't care 🙂 ) https://www.analog.com/media/en/training-seminars/tutorials/mt-001.pdf 

5. IEEE 1241-2010 - IEEE Standard for Terminology and Test Methods for Analog-to-Digital Converters: https://ieeexplore.ieee.org/document/5692956

6. MT-200: Minimizing Jitter in ADC Clock Interfaces:
https://www.analog.com/media/en/training-seminars/tutorials/MT-200.pdf

7. DN1013 - Understanding the Effect of Clock Jitter on High Speed ADCs:
https://www.analog.com/media/en/reference-design-documentation/design-notes/dn1013f.pdf

8. Scientist & Engineer's Guide to Digital Signal Processing, 1999:
https://www.analog.com/en/education/education-library/scientist_engineers_guide.html
(While still largely relevant, note that Chapter 3 on ADC and DAC dwells heavily on high-order analog filters, which have largely been replaced in oversampling converters.)

9. The Data Conversion Handbook, 2005:
https://www.analog.com/en/education/education-library/data-conversion-handbook.html
See Chaper 5, Testing Data Converters. FFT Basics, p. 5.51. Figure 5.67. Talks about choice of prime FFT bin, windowing, other practical stuff.

# Introduction
Simulation of basic FFT operation, SNR calculation
Basically, we're making a vector of N time domain
samples, then corrupting the signal in several "real world" ways:

1. First, start with a perfect signal, or as perfect as a computer can generate. The computer is using at least 64-bit floating point math, which except in extreme cases with multiple error sources, is more than adequate for our purposes.
2.   Quantization noise is added by simply starting with a signal of amplitude
2^(number of bits), then truncating the decimal portion with the int function. We'll take a close look at the resulting errors, and observe qualitatively how closely they resemble typical textbook representations.

2. Develop an understanding of analysis in the frequency domain - how to separate signal power and noise power, calculate SNR in ideal cases.

3. Thermal noise is added to the signal, which is just a random
number with a gaussian distribution. This is often referred to as the
"transition noise" of an ADC, and it can never be smaller than K*T/C where
K is Boltzmann's constant, T is absolute temperature, and C is the size of the
ADC's sample capacitor.

4. Add distortion components to the signal. 

5. Random jitter is added to the sample clock. This is an accurate model of
wideband clock jitter that is spread across seveal Nyquist bands

6. Deterministic jitter is added to the sample clock, representing phase noise
at a particular frequency. This gives an intuitive understanding of the effect
of phase noise as a signal's frequency and amplitude change.

We'll then pick apart the signal in the frequency domain, comparing the power in the signal to "everything else", which we're assuming to be noise.
Shown below is a screenshot from Linear Technology's PScope software, an evaluation tool for ADCs. While not perfect and only targetting proprietary data capture cards, the analysis functions are particularly well thought out. It's worth a half-hour look through the Pscope manual:

**http://ltspice.linear-tech.com/psmanual/manual.pdf**

<img src="pscope_example.PNG" width=600>

# Basics of Analog to Digital conversion: Quantization
Let's start with the very basics. By that - the most fundamental operation of taking a signal that is continuous in both time and value (or amplitude, signal level, whatever you want to call it), and quantizing it in both time and value. One subtle point is that because we're doing this simulation on a digital computer, every single number is already quantized! The default precision of a NumPy floating point number is 64 bits (11 bits exponent, 52 bits mantissa) **(need to confirm)** But this quantization level is MUCH MUCH finer than that of the ADCs that we're simulating, which top out at about 32 bits (and even this is a stretch, which we'll dig into later). This is similar to the idea that when making a precise measurement, your measurement instrument should be some factor (often between four and ten times) more precise than the desired measurement precision.

There are several signal waveforms that we could use for this illustration, a steady (DC) signal, a ramp, a wavelet of some sort, but we're going to use a sinusoid. Most ADCs that are intended for signal acquisition are characterized with sinusoidal inputs. Sinewaves are relatively easy to generate, and easy to analyze using the Fourier transform. We'll be making extensive use of NumPy, and in this case the cosine function:

https://numpy.org/doc/stable/reference/generated/numpy.cos.html

np.cos is simply a function that accepts an array of radian values, and returns an array of the corresponding cosine of those values. There are a few other optional features of the cos function, refer to the documentation.

Enough dragging on here, let's build up the first incarnation of our signal generator, heavily commented :) 



In [ ]:
# Still struggling with getting interactive plots to work. From Travis' notes:
# Try Plotly, Streamlet, Dash.
# ipymply is a dependency for the matplotlib widget backend:
# !pip install ipympl

In [ ]:
"""Define a signal generation function. We'll add more to it later, but at the moment
we want to simply scale the analog signal's range of +/-VREF/2 to +/-2^(bits-1)
The output swings from -2**(bits-1) to +2**(bits-1) for a total span of 2**bits. """

import numpy as np # Import NumPy library

def sine_sig_gen(N, freq, fs, signal_ampl, vref, bits, quantize = True, randomize_phase = False):
    """
    Parameters:
        N - Number of samples
        freq - signal frequency in Hertz
        fs - sample rate in samples per second
        signal_ampl - sinewave signal amplitude (Textbook definition - peak value away from zero)
        vref - ADC voltage reference, representing the peak-to-peak range of inputs
        bits - ADC output will be quantized to 2**bits discrete values
    
    Returns:
        quantized ADC values as floating point array of length N, or unquantized floating point
        values if quantize parameter is set to False.
    """
    smp_times = np.arange(0, N/fs, 1.0/fs)
    """Sampling instants. We want N samples, taken at some sample rate fs in Hertz, which implies 1/fs
    seconds between samples. The NumPy arange function returns evenly spaced values within a given
    interval, which is exactly what we want."""
    
    if randomize_phase:
        phase = np.random.random()* 2.0 * np.pi
    else:
        phase = 0.0
    """Randomize phase if set to True."""
    
    signal = signal_ampl * np.cos(2.0*np.pi*freq*smp_times+phase)
    """Okay, generate the signal. We are using the strict mathematical definition of amplitude, which
    is half of the peak-to-peak excursion of the signal. The cosine function swings from -1.0 to +1.0,
    so no scaling factor is necessary.
    At this point, the signal is discrete in the time domain, because we've got a finite number of samples
    taken over the measurement interval. In the voltage domain, the precision is 64-bit floating
    point, MUCH finer than any real world ADC."""
    
    adc_inf_bits = ((signal / vref) * 2**bits)
    """THIS could be considered the "Fundamental ADC equation". An ADC performs the mathematical operation of
    division - the output code is proportional to the input voltage divided by the reference. Maybe with some
    offset, maybe with some scaling factor, but fundamentally, it's plain old division. The signal is mapped
    to digital values, in this case from -2**(bits-1) to 2**(bits-1). The bits-1 factor is because we've got
    a bipolar signal that swings from negative vref/2 to positive vref/2.
    Note the name of this variable, which implies an "infinity bit ADC". It's not really infinity bits, but
    in this case, a 64-bit float is close enough to infinity that we can assume it's infinity. """

    adc_output = np.around(adc_inf_bits, decimals=0)
    """Here is where the quantization happens! One might be tempted to simply convert to integers at this point, like:
    adc_output = adc_inf_bits.astype(int)
    BUT, there appears to be a subtle effect that knocks off a couple of dB in SNR and produces something that
    looks like odd harmonics. This might be due to how the rounding is done, or the repeated type conversions,
    we'll leave that as an exercise for a motivated reader. So... we'll do this instead! At this point, the signal
    is quantized to within 64-bit floating precision of the "ideal" integer value. """
    
    if quantize:
        return adc_output
    else:
        return adc_inf_bits

In [ ]:

# Okay, let's get started.
# Pull in the good stuff from various libraries...
import numpy as np
from scipy import fft
from matplotlib import pyplot as plt
# from google.colab import output
# output.enable_custom_widget_manager()
#from google.colab import output
#output.enable_custom_widget_manager()


'''############################'''
'''Set up simulation parameters'''
'''############################'''
# There are a few ways that we could express sample rates and frequencies -
# Samples per second for sample rate, Hz for signal frequency, etc. For the sake of
# conveneience let's start by thinking in terms of normalized sample rate of unity (one smample per second, for example)
# Then the FFT output will span from zero (DC) to half the sample rate (or 0.5 cycles per sample)

# If the idea of FFT "bins" is confusing, now might be a good time to do some 
# YouTubing on FFTs. And eventually, we'll get this "deconstructed FFT" moved over to Colab:
# https://github.com/mthoren-adi/ROUS/blob/master/educational/illustrated_fft.py

# But fundamentally - an FFT will take a set of time domain samples and produce an output
# in the frequency domain. Each "bin" represents an EXACT frequency - the first bin (bin 0)
# is zero frequency (DC), the second bin (bin 1) is fs/N, where:
# fs is the sample rate
# N is the number of points in the input data.

fs = 100000.0
""" We're not going to use this just yet, but let's pick a sample rate,
somewhat arbitrarily. We'll use 100ksps because it will comfortably digitize audio,
and when we get to talking to actual hardware, we'll be able to listen to the experiments.
More senses excited = BETTER! """

N = 1024 
""" Number of data points in the input record. The "standard" FFT requires a power of 2
number of data points. This will also be the number of points in the FFT output.
NOTE that you CAN pass a non power of 2"""

bin_number = 4
""" Bin number of the signal itself. If bin_number is greater
than N/2, then the signal will be aliased accordingly.
For now, stick with integer bin numbers. We'll get to non-integer bins soon enough!
Also, prime numbers are less likely to produce repeating artifacts, a subtle but sometimes
noticeable effect."""

freq = fs * bin_number / N

print("Sample Rate: ", fs, " samples per second")
print("FFT Bin Number: ", bin_number)
print("Input frequency: ", fs * bin_number / N, " Hz")

bits = 16
""" ADC resolution (Theoretical!!)
This determines the quantization noise of the converter. 
If you set all of the following imperfections to zero (or close to it),
you should see an SNR of about N * 6.02 + 1.78 dB"""

vref = 5.0
signal_ampl = 0.0625
""" Define a typical reference voltage and signal level to make this a bit more tangible.
Remember that an ADC performs the function of division:
Output code = (input voltage / reference) * 2^bits

Or, if the ADC has scaling / offsetting factors:
Output code = ((input voltage - offset) / full-scale span) * 2^bits

But let's stick with the first one for now.
We'll start out with a 5V reference, and signal level is peak-to-peak such that the ADC
is at full-scale when it's equal to the reference (not RMS, not amplitude, etc.) """

thermal_noise_lvl = 7.6e-6 # 76uV is about 1LSB for a 16-bit ADC w/ 5V reference.
print("thermal noise level (uV): ", thermal_noise_lvl * 1e6)

jitter_lvl = 0.00000001 #0.000025 # clock jitter, expressed as RMS fraction of a sampling interval

# Now for some phase noise... To illustrate the concept, we're going to introduce
# a single tone of phase noise, rather than a distribution (as is the case in
# "real life".) This IS an accurate representation of a sinusoidal disturbance
# on the clock.
phase_noise_offset = 25 # Offset from carrier in bins
phase_noise_lvl = 0.000000001 #.000001 #Amplitude, in fraction of a sample period
'''##############################'''
'''END set up simulation parameters'''
'''##############################'''
print("Parameter Setup Done.")

In [ ]:
%matplotlib widget

adc_output = sine_sig_gen(N, freq, fs, signal_ampl, vref, bits)
signal = sine_sig_gen(N, freq, fs, signal_ampl, vref, bits, quantize = False)

plt.close('all')
plt.figure(1)
fig, ax1 = plt.subplots()

plt.title("Time domain data and exaggerated quantization noise")
plt.xlabel("Sample Number")
plt.ylabel("ADC counts")

ax1.plot(adc_output, color='red')
ax2 = ax1.twinx()
ax2.plot((adc_output - signal), marker='.', color='blue')
ax2.set_ylim(-1.0, 1.0)
ax2.set_ylabel("Quantization Error (lsb)")
fig.tight_layout()
plt.show()


#plt.plot(adc_output)
#plt.plot((adc_output - signal)* 2**(bits-2))
#plt.xlim(0, 100)
#plt.show

# Basics of Signal Analysis using the FFT
We've built up a signal, quantized it, and gotten a feel for what the error, or noise, looks like in the time domain. But how does quantization noise manifest in the frequency domain, and how do we start picking apart the spectrum and comparing signal to noise?

In [ ]:
# First, let's set up our parameters to accentuate the effects we're looking for, or at least make them easier to locate:
N = 4096
fs = 100000
bin_number = 103
signal_ampl = 2.5
vref = 5.0
bits = 16

freq = fs * bin_number / N


adc_output = sine_sig_gen(N, freq, fs, signal_ampl, vref, bits)

print("length: ", len(adc_output))
print("max value: ", np.max(adc_output))
print("min value: ", np.min(adc_output))
print("DC content: ", np.average(adc_output))

freq_domain = np.fft.fft(adc_output)
freq_domain_magnitude = np.abs(freq_domain)/N

freq_domain_signal = np.zeros(N)
freq_domain_noise = np.zeros(N)

#Now notch the signal out of the spectrum. We have the advantage here
#that there's only a single bin of signal, and no distortion.

np.copyto(freq_domain_noise, freq_domain_magnitude) #Make a copy
freq_domain_noise[bin_number] = 0 #Zero out positive signal bin
freq_domain_noise[N - bin_number] = 0 # And the negative bin
# Note that we're also zeroing out one bin worth of noise. We're going to assume this is insignificant
# in this simulation, but if you're zeroing out lots of bins with a mask, you might want to fill them in
# with the average noise floor from the bins that aren't zeroed
# (or some more intelligent estimate if the noise floor is wiggling around.)

#Make another array that just has the signal
freq_domain_signal[bin_number] = freq_domain_magnitude[bin_number]
freq_domain_signal[N - bin_number] = freq_domain_magnitude[N - bin_number]

signal = 0.0 #Start with zero signal, zero noise
noise = 0.0

# Sum the power root-sum-square in each bin. Abs() function finds the power, a resistor dissipating
# power does not care what the phase is!

signal_ss = np.sum(((freq_domain_signal)) ** 2)
noise_ss = np.sum(((freq_domain_noise)) ** 2 )

signal = np.sqrt(signal_ss)#  / N
noise = np.sqrt(noise_ss) # / N

snr_fraction = signal / noise
snr = 20*np.log10(signal / noise)
print ("Signal: " + str(signal))
print ("Noise: " + str(noise))
print ("Fractional signal to noise: " + str(snr_fraction))
print ("SNR: " + str(snr) + "dB")
print ("Theoretical SNR based on 6.02*bits + 1.76: ", (6.02 * bits) + 1.76)

max_freq_domain_magnitude = max(freq_domain_magnitude)
min_mag = np.ones(N)* 1e-14
freq_domain_magnitude_db = 20 * np.log10(np.maximum(freq_domain_magnitude, min_mag) / max_freq_domain_magnitude)

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()

%matplotlib widget
plt.close('all')
plt.figure(1)
plt.title("Time domain data, with imperfections")
plt.xlabel("Sample Number")
plt.ylabel("ADC counts")
plt.plot(adc_output)
#plt.xlim(0, 100)
plt.show

plt.figure(2)
plt.title("Spectrum")
plt.xlabel("FFT Bin")
plt.ylabel("Signal Level, dBC")
plt.plot(freq_domain_magnitude_db)
# plt.xlim(0, 500) # Uncomment to manually zoom
#plt.ylim(-140, 0)
plt.show

Alrighty, that was a nice warmup! we've compared a single bin worth of signal to the rest of bins as noise, and plotted out the FFT. But why is the FFT output "mirrored"? Remember that the output of an FFT has the same number of points as the input data, and bin 1 represents a frequency of fs/N. Visually, this is a sinewave that covers a single cycle over the entire time record. And from Nyquist, the FFT output can only digitize to fs/2, which is the center of the FFT. The right-hand side of the FFT is thus a mirroring of the left side, from fs/2 back to DC. Sort of. It's more accurate to think of it as INCREASING in frequency from fs/t UP to fs! Remember that a signal at fs... will ALIAS to DC!

Another important concept - both the input and output of an FFT are complex. It just happens that if you only stuff real values into the FFT input, the outputs will be complex conjugates mirrored around fs/2. That's a mouthful, so let's take a look at a couple of examples from the data above:

In [ ]:
# Pick a bin, any bin (from 1 to N/2)
test_bin = 3
print(freq_domain[test_bin])
print(freq_domain[N-test_bin])
print("Check it out... real parts should be equal, imaginary equal and opposite sign...")

With that out of the way, let's button up our method of plotting FFTs - when we're working with only real samples (vs. complex, as in an I/Q radio signal), we can simply look at bins 0 to N/2-1, and double each bin. Any real valued signal that exceeds fs/2 will simply alias back into the range of DC to fs/2.

In [ ]:
""" First, let's Button up a few utility functions. """

import numpy as np



def plot_time_and_freq(t, f):
    """Basic plotting routine for time and frequency"""
    from matplotlib import pyplot as plt
    plt.close('all')
    plt.figure(1)
    plt.title("Time domain data, with imperfections")
    plt.xlabel("Sample Number")
    plt.ylabel("ADC counts")
    plt.plot(t)
    #plt.xlim(0, 100)
    plt.show

    plt.figure(2)
    plt.title("Spectrum")
    plt.xlabel("FFT Bin")
    plt.ylabel("Signal Level, dBC")
    plt.plot(f)
    plt.xlim(0, 100) # Uncomment to manually zoom
    plt.show

def calculate_spectrum_dbc(data, ref = None, dBc = True):
    """ Calculate spectrum in dB releative to the carrier. """
    freq_domain = np.fft.fft(data)
    freq_domain_magnitude = (2.0*np.abs(freq_domain)/len(data))[:len(data)//2]
    max_freq_domain_magnitude = np.max(freq_domain_magnitude)
    if ref is None:
        ref = max_freq_domain_magnitude
    freq_domain_magnitude_db = 20 * np.log10(freq_domain_magnitude / ref)
    return freq_domain_magnitude_db



# Windowing
Although it might not have seemed significant at first glance, the biggest simplification in the introductory example is that we parked the fundamental signal at an exact FFT bin. In fact - we DEFINED the signal frequency in terms of FFT bin, then calculated what its actual frequency was based on the sample rate and number of data points.

In reality - this situation is surprisingly difficult to achieve in real life: it requires two signal sources - one to set the sample rate (often called the "clock") and another to generate the analog input signal. They both must be stable, and at least one of them must have very fine frequency resolution (we'll define HOW fine shortly...). Furthermore, they either need to be extraordinarily accurate (like, rubidium frequency standard accurate), OR more realistically (and lower cost), both sharing the same frequemcy reference. Often benchtop synthesizers will have a 10MHz reference input, output, or both, as BNC jacks on the back panel.

What happens if you're not on an exact frequency bin? Well, you end up with a partial cycyle of the fundamental signal in the data record, and the fundamental "leaks" all over the spectrum. Let's get a picture of how bad it can get:

https://www.gaussianwaves.com/2020/09/window-function-figure-of-merits/

In [ ]:
""" Do some experimentation here - let's leave the bin number at 50.0050,
    which is a 100ppm departure from exactly bin 50. Try 1ppm departures, 1000ppm, etc.
    to get a feel for just how perfect you need to be to get an accurate measurement
    of the fundamental with no leakage.
    
    Also note that the phase is randomized so you should be able to run this cell
    over and over again, and see how the leakage "dances" around :)
    """

#time_domain = create_signal(4096, 50.0050, 16, 1.0, 1.0, 0.00001, 0.0, 0.0, 0.0, randomize_phase=True)

N = 4096
fs = 100000
bin_number = 50.05
signal_ampl = 2.5
vref = 5.0
bits = 16

freq = fs * bin_number / N

time_domain = sine_sig_gen(N, freq, fs, signal_ampl, vref, bits, randomize_phase = True)

print("Here's a few samples: ", time_domain[0:8])

freq_domain_magnitude_db = calculate_spectrum_dbc(time_domain, ref = 2 ** 16)

print("Here's a few bins: ", freq_domain_magnitude_db[48:53])

plot_time_and_freq(time_domain, freq_domain_magnitude_db)

In [ ]:
""" So how the heck do we mitigate this effect? the answer is "windowing".
    """

N = 4096
fs = 100000
bin_number = 50.05
signal_ampl = 2.5
vref = 5.0
bits = 16

freq = fs * bin_number / N

time_domain = sine_sig_gen(N, freq, fs, signal_ampl, vref, bits, randomize_phase = True)

window = np.blackman(len(time_domain))

windowed_time_domain = time_domain * window

freq_domain_magnitude_db = calculate_spectrum_dbc(windowed_time_domain, ref = 2 ** 16)

print("Here's a few bins: ", freq_domain_magnitude_db[48:53])

plot_time_and_freq(windowed_time_domain, freq_domain_magnitude_db)

**Deeper Analysis**
Before digging into how to calculate SNR, THD, etc. let's look at a couple of "pre-canned" tools. PScope is Analog Device's ADC testing tool, which while it only talks to proprietary hardware, does give you the ability to load data and analyze it. The file format is somewhat proprietary, so the "save_for_pscope" library takes care of formatting. Another library, sin_params, is a Python implementation of many of PScope's analysis functions. After running the following cell, open Pscope, then file -> Load Data, and open time_domain.adc.

Notice that Pscope does not apply a window by default - select Blackman-Harris 92dB, then compare parameters.

**Note** - looks like there may be a bug, SNR from sin_params looks like it might actually be THD.

Screenshot:

<img src="pscope_vs_sin_params.PNG" width=800>

In [ ]:
from save_for_pscope import save_for_pscope
from sin_params import *

# Saving time_domaine twice - workaround because PScope expects two channels
save_for_pscope("time_domain.adc", 16, True, len(time_domain), "DC0000", "LTC1111", time_domain, time_domain, )

harmonics, snr, thd, sinad, enob, sfdr, floor = sin_params(time_domain)
print("A.C. Performance parameters (ONLY valid for a sine input):")
print("Harmonics:", harmonics)
print("snr: ", thd)
print("Sinad: ", sinad)
print("ENOB: ", enob)
print("SFDR: ", sfdr)
print("Noise Floor: ", floor)


# Thermal Noise
So far, the only imperfections we've introduced are quantization noise (which is actually a property of the signa chain) and non-coherent effects (which are mathematical artifacts.) Now we're going to add in thermal noise, which will ALWAYS exist in any signal. In fact, let's start with a useful simplification and assume that our signal chain is absolutely perfect, and the ADC's sampling circuit is the ONLY source of noise. Many ADC architectures have a sample and hold circuit that captures a sample on a small capacitor with a network of switches as shown below:

**To Do: Add LTspice pic**

The very act of capturing a sample on a capacitor, even a perfect one, introduces noise. If you repeatedly capture a noiseless signal on a capacitor of size C, then the resulting samples will NOT be perfect; they will have a distribution of voltages with a standard deviation of:

SQRT(K*T/C)

Where:

K = Boltzmann's constant (1.380649 × 10-23 J/K)
T = absolute temperature in Kelvin
C = capacitance in farads.

Similar to an ADC's "number of bits", the sample capacitor (along with the full-scale input range) represents an UPPER LIMIT on an ADC's Signal to Noise ratio.

In [ ]:
"""Add thermal noise to the signal generation function """

import numpy as np # Import NumPy library

def sine_sig_gen(N, freq, fs, signal_ampl, vref, bits, t_noise, quantize = True):
    """
    Parameters:
        N - Number of samples
        freq - signal frequency in Hertz
        fs - sample rate in samples per second
        signal_ampl - sinewave signal amplitude (Textbook definition - peak value away from zero)
        vref - ADC voltage reference, representing the peak-to-peak range of inputs
        bits - ADC output will be quantized to 2**bits discrete values
        t_noise - RMS thermal noise level
    
    Returns:
        quantized ADC values as floating point array of length N, or unquantized floating point
        values if quantize parameter is set to False.
    """
    smp_times = np.arange(0, N/fs, 1.0/fs) # Sampling Instants    
    signal = signal_ampl * np.cos(2.0*np.pi*freq*smp_times) # Generate the signal
    noise = np.random.normal(loc=0, scale=t_noise, size=N)
    signal += noise # Add noise. (Nothing's perfect :( ))
    adc_inf_bits = ((signal / vref) * 2**bits) # Scale to number of output codes
    adc_output = np.around(adc_inf_bits, decimals=0) # Quantize
    
    if quantize:
        return adc_output
    else:
        return adc_inf_bits

**To Cover Next:**


*   Different types of windows and their figures of merit
*   Interpreting windowed FFT data - how to sum power across a few bins, etc.
*   SNR/THD calculations, locating harmonics, figuring out what to count as signal, noise, and harmonics.
*   Masking out regions of the noise floor that we know can't be blamed on the ADC (automasking in Pscope)
*   Video (scalar) averaging to smooth out the noise floor
*   Vector averaging to reduce noise floor while accurately measuring harmonically related components



# Talking to Hardware (Warmup)

In [ ]:
# Okay now let's talk to some hardware!

# Here's how to connect to a local runtime:
# https://research.google.com/colaboratory/local-runtimes.html
# And after doing so, this actually works!!:
# For reference:
# pip install --upgrade jupyter_http_over_ws>=0.0.7 && jupyter serverextension enable --py jupyter_http_over_ws

# jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com'  --port=8888  --NotebookApp.port_retries=0


# It looks like installing from pip works fine, but...
# shouldn't need to do here if the Jupyter environment is set up properly.
# !pip install pyadi-iio

import adi
print("ADI path:")
print(adi.__file__)
print("ADI objects:")
print(dir(adi))



In [ ]:
# Warmup for seeing if we can talk to hardware here...
# Note that you need to have a Pi with an LM75 attached somewhere on your network
my_uri = "ip:experimenter.local" # "experimenter" is Thoren's "always-on" hacker board
my_temp_sensr = adi.lm75(uri=my_uri)

print("\nChecking temperature channel...")
print("Temperature raw: " + str(my_temp_sensr.input))
print(
    "Temperature in deg. Celsius: " + str(my_temp_sensr.to_degrees(my_temp_sensr.input))
)

print("\nUpdate interval: " + str(my_temp_sensr.update_interval))


print("\nMax threshold: " + str(my_temp_sensr.to_degrees(my_temp_sensr.max)))
print("Max hysteresis: " + str(my_temp_sensr.to_degrees(my_temp_sensr.max_hyst)))

print("\nSetting max threshold, hyst. to 30C, 25C...\n")

my_temp_sensr.max = my_temp_sensr.to_millidegrees(30.0)
my_temp_sensr.max_hyst = my_temp_sensr.to_millidegrees(25.0)

print("New thresholds:")
print("Max: " + str(my_temp_sensr.to_degrees(my_temp_sensr.max)))
print("Max hysteresis: " + str(my_temp_sensr.to_degrees(my_temp_sensr.max_hyst)))

del my_temp_sensr

# Talkigng to Hardware (Actual ADC data)

Okay, now that we've successfully talked to a basic sensor, let's grab some data from an actual ADC!

In [ ]:
'''
NOTE: At the moment, the LTC2387 interface is in this branch:
https://github.com/mthoren-adi/pyadi-iio/tree/ltc2387_dev

'''


import numpy as np
from scipy import signal

device_name = "ltc2387"

# Optionally passs URI as command line argument,
# else use default context manager search
my_uri = sys.argv[1] if len(sys.argv) >= 2 else None
print("uri: " + str(my_uri))

my_adc = adi.ltc2387(uri=my_uri)
my_adc.rx_buffer_size = 4096
my_adc.sampling_frequency = 10000000

data = my_adc.rx()

# Figure out how to do this correctly - need to sign extend bit 17
for i in range(len(data)):
    if data[i] > 2 ** 17:
        data[i] -= 2 ** 18

x = np.arange(0, len(data))
plt.figure(1)
plt.clf()
plt.plot(x, data)
plt.show()

f, Pxx_den = signal.periodogram(data, my_adc.sampling_frequency, window="blackman")

plt.figure(2)

plt.clf()
plt.semilogy(f, Pxx_den)
plt.ylim([1e-5, 1e5])
plt.xlabel("frequency [Hz]")
plt.ylabel("PSD [V**2/Hz]")
plt.draw()
plt.pause(0.05)

del my_adc

Next Steps:
1. Run sin_params analysis
2. Pick apart analysis, cover windowing, summing signal, noise, and harmonic power